In [1]:
#================================================================
# Evaluator
#================================================================
from cognify.optimizer.evaluation.metric import F1Str

metric = F1Str()

In [2]:
#================================================================
# Data Loader
#================================================================

import json

def load_data_minor():
    with open("data._json", "r") as f:
        data = json.load(f)
          
    # format to (input, output) pairs
    new_data = []
    for d in data:
        input = (d["question"], d["docs"])
        output = d["answer"]
        new_data.append((input, output))
    return new_data[:5], None, new_data[5:]

In [4]:
#================================================================
# Optimizer Set Up
#================================================================
from cognify.cog_hub import default_search

search_settings = default_search.create_search()

/mnt/ssd4/lm_compiler/my_env/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [5]:
train, val, test = load_data_minor()

In [5]:
import cognify.run.optimize

opt_cost, pareto_frontier, opt_logs = cognify.run.optimize.optimize(
    script_path="cognify_workflow.py",
    control_param=search_settings,
    train_set=train,
    val_set=val,
    eval_fn=metric,
)

[INFO 2024-11-17 04:40:22] Loading light_opt_layer params from opt_results/light_opt_layer/opt_params.json


> light_opt_layer | (best score: 0.00, lowest cost@1000: 0.00 $):  50%|#####     | 10/20 [00:00<?, ?it/s]

---> Evaluation in light_opt_layer_10 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<…

---> Evaluation in light_opt_layer_11 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<…

---> Evaluation in light_opt_layer_12 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<…

...

---> Evaluation in light_opt_layer_17 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<…

---> Evaluation in light_opt_layer_18 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<…

---> Evaluation in light_opt_layer_19 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/5 [00:00<…

[INFO 2024-11-17 04:40:40] ----------------- Optimization Finished -----------------


=========== Optimization Results ===========
Num Pareto Frontier: 2
--------------------------------------------------------
Pareto_1
  Quality: 0.418, Cost per 1K invocation ($): 0.17
  Applied Optimization: light_opt_layer_12
--------------------------------------------------------
Pareto_2
  Quality: 0.420, Cost per 1K invocation ($): 0.19
  Applied Optimization: light_opt_layer_19


In [5]:
from cognify.run.evaluate import evaluate, load_workflow

In [7]:
new_workflow = load_workflow(config_id='Pareto_1', opt_result_path='opt_results')

In [8]:
input = (
    "What was the 2010 population of the birthplace of Gerard Piel?", 
    [
        'Gerard Piel | Gerard Piel (1 March 1915 in Woodmere, N.Y. – 5 September 2004) was the publisher of the new Scientific American magazine starting in 1948. He wrote for magazines, including "The Nation", and published books on science for the general public. In 1990, Piel was presented with the "In Praise of Reason" award by the Committee for Skeptical Inquiry (CSICOP).',
        'Woodmere, New York | Woodmere is a hamlet and census-designated place (CDP) in Nassau County, New York, United States. The population was 17,121 at the 2010 census.',
    ],
)

new_workflow(input)

'The population of Woodmere, New York in 2010 was 17,121.'

In [9]:
eval_result = evaluate(
    config_id='Pareto_1',
    test_set=test,
    opt_result_path='opt_results',
    n_parallel=10,
    eval_fn=metric,
    save_to='eval_results.json',
)

----- Testing select trial light_opt_layer_12 -----
  Params: {'qa_agent_few_shot': 'NoChange', 'qa_agent_reasoning': 'NoChange'}
  Quality: 0.418, Cost per 1K invocation ($): 0.17 $


> Evaluation in light_opt_layer_12 | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/10 [00:00<?,…

=========== Evaluation Results ===========
  Quality: 0.453, Cost per 1K invocation ($): 0.16 $


In [10]:
from cognify.run.inspect import inspect

In [11]:
inspect(
    opt_result_path='opt_results'
)

[INFO 2024-11-17 04:46:50] Loading existing dry run result at opt_results/dry_run_train.json


=========== Optimization Results ===========
Num Pareto Frontier: 2
--------------------------------------------------------
Pareto_1
  Quality: 0.418, Cost per 1K invocation ($): 0.17
  Applied Optimization: light_opt_layer_12
--------------------------------------------------------
Pareto_2
  Quality: 0.420, Cost per 1K invocation ($): 0.19
  Applied Optimization: light_opt_layer_19
